In [ ]:
import sys
sys.path.append("/home/missinguser/CSE/single-stage-opt/hybrid_tokamak/laptop/")
import latexplot
latexplot.set_cmap(4)
import os
from typing import Literal
import numpy as np
import matplotlib.pyplot as plt
import simsopt
from simsopt import mhd
from simsopt import geo
import json
from scipy.spatial.distance import cdist
from scipy.stats import linregress
from scipy.io import netcdf_file
from scipy.optimize import curve_fit
import scipy.ndimage.filters
import warnings
from pathlib import Path
import asyncio
from concurrent.futures import ThreadPoolExecutor
from simsopt.configs import get_QUASR_data
from simsopt.mhd.vmec_diagnostics import vmec_compute_geometry

sys.path.append("/home/missinguser/CSE/single-stage-opt/")
sys.path.append("/home/missinguser/CSE/regcoil")
from replicate_lgradb.find_single_l import find_regcoil_distance


MAX_SVD_VALS = 40
SINGLE_STAGE_PATH = Path.home() / "single-stage-opt/"


def coil_surf_distance(curves, lcfs) -> np.ndarray:
    pointcloud1 = lcfs.gamma().reshape((-1, 3))
    distances = [np.min(cdist(pointcloud1, c.gamma()), axis=0) for c in curves]
    return np.array(distances).T


def compute_coil_surf_dist(simsopt_filename):
    surfaces, coils = simsopt.load(simsopt_filename)
    lcfs = surfaces[-1].to_RZFourier()

    curves = [c.curve for c in coils]
    return coil_surf_distance(curves, lcfs)


def fit_exponential_rate(sequence, title):
    x = np.linspace(0, 1, len(sequence))

    # log-linear fit 
    fit = np.polyfit(x, np.log(sequence), 1)
    a_initial = sequence[0]
    b_initial = fit[0]
    return b_initial

    # Exponential curve fit using log linear as initial guess
    # Define the exponential function for fitting
    def exp_model(x, a, b, c):
        return a * np.exp(b * x) + c

    try:
        # Perform the nonlinear fit using the initial guesses
        popt, pcov = curve_fit(exp_model, x, sequence, p0=[a_initial, b_initial, 0])

        # Extract fitted parameters
        a_fit, b_fit, _ = popt
    except:
        return np.nan

    print(f"linear={b_initial:.2f}, exponential={b_fit:.2f}")
    # plt.semilogy(x, sequence)
    # fit = np.polyfit(x, np.log(sequence), 1)
    # plt.semilogy(x, np.exp(np.polyval(fit, x)))
    # plt.title(title)
    # plt.show()
    if b_fit > 100 or b_fit < -100:
        return np.nan
    return b_fit


def vs_plot(x_data, y_data, labels=None):
    x_vals, x_label = x_data
    y_vals, y_label = y_data
    title = x_label + " vs " + y_label

    if len(np.shape(y_vals)) >= 2:
        y_vals = np.array(y_vals).T
    elif len(np.shape(y_vals)) == 1:
        y_vals = np.reshape(y_vals, (1,) + np.shape(y_vals))

    assert len(x_vals) == len(y_vals[0])
    if labels is not None:
        assert len(labels) == len(y_vals)

    # Linear fit
    # TODO this Fails because some values are inf!!
    for i, y in enumerate(y_vals):
        plt.scatter(x_vals, y, label=title if labels is None else labels[i], s=4)
        reg = linregress(x_vals, y)
        plt.axline(
            xy1=(0, reg.intercept),
            slope=reg.slope,
            color="k" if len(y_vals) == 1 else plt.rcParams["axes.prop_cycle"].by_key()["color"][i],
            label=f"Linear fit {i}: $R^2$ = {reg.rvalue**2:.3f}",
        )

    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    if np.min(y_vals) >= 0:
        plt.gca().set_ylim(bottom=0)
    plt.gca().set_xlim(left=0)
    plt.grid(True)
    plt.legend()


In [ ]:

regcoil_plot = True
bdistrib_plot = True


import pandas as pd
df:pd.DataFrame = pd.read_pickle("../quasr_coil_check/QUASR_db/QUASR_08072024.pkl")

# Filter df by constant number of coils
df = df[df["nc_per_hp"] * df["nfp"] == 6]
df = df.sample(n=500, replace=False, random_state=42)
print(df.columns)
print(df["ID"])


LgradB_keyed = {}
coil_surf_dist = {}

desc_outputs = list(
    filter(lambda x: x.endswith("_output.h5"), os.listdir()))
ids = df["ID"].tolist()

vmec = mhd.Vmec(verbose=False)
# vmec.indata.mpol = 6
# vmec.indata.ntor = 5
# vmec.indata.niter_array[:3] = [1000,2500, 4000]
# vmec.indata.ns_array[:3] = [6, 13, 31]
# vmec.indata.ftol_array[:3] = [1e-5, 1e-7, 1e-10]


def get_QUASR_data_sync(idx, style='quasr-style'):
    # Your original synchronous function
    data = get_QUASR_data(idx, style)
    return idx, data[0], data[1]

async def fetch_data(idx, loop, executor):
    surfs, coils = await loop.run_in_executor(executor, get_QUASR_data_sync, idx)
    return idx, surfs, coils
    # try:
    # # Network error
    # except ValueError as e:
    #     print(f"Network error with {idx}: {e}")
    #     return idx, None, None

# loop = asyncio.get_event_loop()
# executor = ThreadPoolExecutor()
# tasks = [fetch_data(idx, loop, executor) for idx in ids]
# results = await asyncio.gather(*tasks)
results = map(get_QUASR_data_sync, ids)
results = list(filter(lambda x: x[1] is not None, results))
# loop.stop()
# executor.shutdown()

## Compute bdistrib for these configs (from online DB)
Save them to the local database, then call `python -m quasr_coil_check.precompute_bdistrib` to compute the bdistribs.

In [ ]:
def normalize_scale(surface, constant_minor=True):
  # Scaling factor for either constant minor or major radius
  if constant_minor:
    scaling = 1.704 / surface.minor_radius()
  else: 
    scaling = 1.0 / surface.major_radius()
  return scaling

In [ ]:
from quasr_coil_check import bdistrib_io
from quasr_coil_check import precompute_surfaces

for res in results:
    i, coils, surfs = res
    # Write to disk to use backwards compatible file format
    # print(i, coils, surfs)
    surfs_scaled = []
    for surf in surfs:
        scaling = normalize_scale(surf)
        surfs_scaled.append(surf.copy())
        surfs_scaled[-1].rc_array *= scaling
        surfs_scaled[-1].zs_array *= scaling
    
    path = Path(bdistrib_io.get_file_path(i, "simsopt"))
    path.parent.mkdir(parents=True, exist_ok=True)
    simsopt.save((coils, surfs_scaled), path)


In [ ]:

from joblib import Memory
location = './.cachedir'
memory = Memory(location, verbose=0)


def lgradbsc(computed, idx):
    gradB = np.array([[computed.grad_B__XX, computed.grad_B__YX, computed.grad_B__ZX],
                      [computed.grad_B__XY, computed.grad_B__YY, computed.grad_B__ZY],
                      [computed.grad_B__XZ, computed.grad_B__YZ, computed.grad_B__ZZ]])
    scalarGradB = np.sqrt(gradB[0,0] **2 + gradB[1,1]**2 + gradB[2,2] **2)
    LgradBs = (
        np.sqrt(2)
        * computed.modB
        / np.linalg.norm(gradB, ord="fro", axis=(0, 1))
    )
    LgradBscalar = (
        computed.modB / scalarGradB
    )
    # "$L_{\\max \\sigma}$"
    LgradB2maxsigma = (
        computed.modB
        / np.linalg.norm(gradB, ord=2, axis=(0, 1))
    )
    # "$L_{\\sigma}$"
    # TODO: Not entirely sure if the authors meant the nuclear norm (sum of singular values) or this norm (root mean square of singular values)
    LgradB2sigma = (
        np.sqrt(2)
        * computed.modB.flatten()
        # / np.linalg.norm(gradB, ord="nuc", axis=(0, 1))
        / np.sqrt(np.sum(np.linalg.svd(gradB.reshape((3,3,-1)).T, compute_uv=False)**2, axis=1))
    )
    Lgradbgradb = (
        computed.modB * computed.modB / np.linalg.norm(np.sum(np.array([computed.B_X, computed.B_Y, computed.B_Z]) * gradB, axis=0), axis=0)
    )
    #  ["$L^*_{\\nabla \\vec{B}}$", 
    #   "$L_{\\vec{B} \\cdot \\nabla \\vec{B}}$", 
    #   "$L_{\\nabla |B|}$", 
    #   "$L_{\\max \\sigma}$",
    #   "$L_{\\sigma}$"]
    return  np.array([ np.min(LgradBs), np.min(Lgradbgradb), np.min(LgradBscalar), np.min(LgradB2maxsigma), np.min(LgradB2sigma)])

@memory.cache(ignore=["vmec"])
def vmec_lgradbsc(vmec, idx):
    try:
        vmec.run()
        s = [1]
        ntheta = 64
        nphi = 64
        theta = np.linspace(0, 2 * np.pi, ntheta, endpoint=False)
        phi = np.linspace(0, 2 * np.pi / vmec.boundary.nfp, nphi, endpoint=False)
        data = vmec_compute_geometry(vmec, s, theta, phi)
        B0 = np.mean(data.modB)
        print("B0", B0)
        vmec.indata.phiedge = 5.865 * vmec.indata.phiedge / B0
        vmec.recompute_bell()
        vmec.run()
        computed = vmec_compute_geometry(vmec, s, theta, phi)
        return lgradbsc(computed, idx)
    except Exception as e:
        print(f"VMEC didn't converge! Error with {idx}: {str(e)}")
    return np.array([0.0, 0.0, 0.0, 0.0, 0.0])

In [ ]:

def compare_bdistrib(idx):
    filepath = bdistrib_io.get_file_path(idx, "bdistrib")
    # Some fits of the exponential decay, take a look at compareSingularValuesPlot.py and bdistrib_util.py
    
    bdistrib_variables = [
        "Bnormal_from_const_v_coils_inductance",
        "Bnormal_from_const_v_coils_transfer",
        # "svd_s_transferMatrix",
        # "svd_s_inductance_plasma_middle",
        # "svd_s_inductance_plasma_middle",
    ]
    
    fit_types = [
        "log_linear",
        #  "value",
         "windowed_upper_bound"
    ]
    fits = {}
    
    if not Path(filepath).exists():
        fits = {}
        for key in bdistrib_variables:
            for fit_type in fit_types:
                fits[key + fit_type] = np.nan
        return fits

    with netcdf_file(filepath, "r", mmap=False) as f:
        for key in bdistrib_variables:
            vararray = f.variables[key][()].flatten()
            vararray = np.abs(vararray)  # [:MAX_SVD_VALS]
            for fit_type in fit_types:
                if fit_type == "value":
                    fit_val = np.max(np.log(vararray))
                elif fit_type == "log_linear":
                    # Only take the 0 element (slope) of the fit
                    fit_val = fit_exponential_rate(vararray, key)
                else:
                    windowed_array = scipy.ndimage.filters.maximum_filter1d(
                        vararray, size=16
                    )
                    fit_val = fit_exponential_rate(windowed_array, key)
                fits[key + fit_type] = fit_val
    return fits

In [ ]:
vmecs = {}
regcoil_distances = {}
num = 65 # start at 65 because the kernel keeps dying???
for idx, surfs, coils in results[num:]:
    num += 1
    print(num)
    s : geo.SurfaceRZFourier = surfs[-1].to_RZFourier().copy()

    # If the configurations are all scaled to the same major radius instead of minor radius, the results that follow are not qualitatively different
    scaling = normalize_scale(s)
    s.rc_array *= scaling
    s.zs_array *= scaling
    print(s.major_radius())

    vmec.boundary = s
    vmec.indata.nfp = vmec.boundary.nfp
    # print(idx)
    vmecs[idx] = vmec_lgradbsc(vmec, idx)

    # print(vmecs[idx])
    if np.allclose(vmecs[idx], 0): 
        regcoil_distances[ f"{idx:07d}" ] = 0
    else:
        # Must pass 
        regcoil_distances[ f"{idx:07d}" ] = find_regcoil_distance(vmec, idx)
    print("Lregcoil", regcoil_distances[ f"{idx:07d}" ])

In [ ]:
import matplotlib
cmap10 = False
if __name__ == "__main__":
    bdistrib_plot = True
    latexplot.set_cmap(4)
    if cmap10: 
        plt.rcParams['axes.prop_cycle'] = matplotlib.cycler(color=[plt.get_cmap('tab10')(e) for e in range(4)])

    lgradB_names = ["$L^*_{\\nabla \\vec{B}}$", 
                    "$L_{\\vec{B} \\cdot \\nabla \\vec{B}}$", 
                    "$L_{\\nabla |B|}$", 
                    "$L_{\\max \\sigma}$",
                    # "$L_{\\sigma}$",
                    ]
    
    for idx, surfs, coils in results:
        filename =  f"{idx:07d}" 
        print("Loading", filename)
        vmec.boundary = surfs[-1]
        vmec.indata.nfp = vmec.boundary.nfp
        try:
            computed = vmecs[idx]
        except simsopt._core.util.ObjectiveFailure as e:
            print(f"VMEC didn't converge! Error with {idx}: {e}")
            continue 
        except KeyError:
            # VMEC didn't converge for this configuration, so its not a valid result
            continue

        # $L_{REGCOIL}$
        simsopt_name = filename.replace("input.", "serial").replace(
            "_output.h5", ".json"
        )
        LgradB_keyed[simsopt_name] = vmecs[idx]
        LgradB = LgradB_keyed[simsopt_name][0]

        # The distances here were verified with the QUASR database GUI and are correct.
        # simsopt_path = f"{SINGLE_STAGE_PATH}/quasr_coil_check/QUASR_db/simsopt_serials/{simsopt_name[6:10]}/{simsopt_name}"
        coil_surf_dist[simsopt_name] = coil_surf_distance([c.curve for c in coils], surfs[-1]) * normalize_scale(surfs[-1]) #compute_coil_surf_dist(simsopt_path) 
        
        print(
            simsopt_name,
            "has minimum filament coil distance",
            np.min(coil_surf_dist[simsopt_name]),
        )

        # Only plot all individual equilibria for small datasets
        if LgradB < 0.05 and False:
            latexplot.set_cmap(len(coils))
            latexplot.figure()
            cross = vmec.boundary.cross_section(0, np.linspace(0, 1, 128, endpoint=True))
            plt.subplot(2, 2, 1)
            plt.plot(cross[:,0], cross[:,2], label=f"{filename}  LgradB={LgradB}")
            cross = vmec.boundary.cross_section(np.pi/2, np.linspace(0, 1, 128, endpoint=True))
            plt.plot(cross[:,1], cross[:,2], label=f"{filename}  LgradB={LgradB}")
            plt.subplot(2, 2, 2)
            # plt.imshow(LgradB_keyed[simsopt_name][-1]) 
            # plt.colorbar()
            plt.subplot(2, 2, 4)
            # plt.imshow(LgradB_keyed[simsopt_name][0]) 
            # plt.colorbar()


            # desc.plotting.plot_2d(eq, "L_grad(B)", ax=ax[0, 1], cmap="jet_r")
            # desc.plotting.plot_2d(eq, "L_grad(B)", ax=ax[0, 1], cmap="jet_r")

            if regcoil_plot:
                plt.hlines(
                    LgradB,
                    0,
                    1,
                    linestyles="dashed",
                )
                plt.hlines(
                    regcoil_distances[filename], 
                    0,
                    1,
                )
                plt.ylim(bottom=0)
                plt.legend(
                    ["$L^*_{\\nabla B}$", "Coil winding surf. dist."])

            # QUASR Filament coil distance
            plt.subplot(2, 2, 3)
            plt.plot(coil_surf_dist[simsopt_name], label="coil")
            plt.hlines(
                LgradB, 0, coil_surf_dist[simsopt_name].shape[0], linestyles="dashed"
            )
            plt.title("Filament coil distance")
            plt.legend()
            plt.ylim(bottom=0)

            plt.suptitle(filename)
            latexplot.savenshow(filename)

    #########################

    # Extract filenames and corresponding values for plotting
    latexplot.set_cmap(len(lgradB_names))
    if cmap10: 
        plt.rcParams['axes.prop_cycle'] = matplotlib.cycler(color=[plt.get_cmap('tab10')(e) for e in range(4)])
    validlgradb_dict = {key: value for key, value in LgradB_keyed.items() if np.all(value > 0.0)} #
    filenames = list(validlgradb_dict.keys())
    if regcoil_plot:
        validr_dict = {key: value for key, value in regcoil_distances.items() if (value != 0) and np.isfinite(value)}
        bothLgradBandRegcoil = list(set(validlgradb_dict.keys()).intersection(validr_dict.keys()))
        regcoil_vals = (np.array([regcoil_distances[f] for f in bothLgradBandRegcoil]), "$L_{REGCOIL}$")
        
        LgradB_vals = (np.array([LgradB_keyed[f][:len(lgradB_names)] for f in bothLgradBandRegcoil]), "$L^*_{\\nabla B}$") 
        coil_min_vals = (
            [np.min(coil_surf_dist[f]) for f in bothLgradBandRegcoil],
            "QUASR coil distance",
        )
        latexplot.figure()
        vs_plot(regcoil_vals, LgradB_vals, lgradB_names)
        latexplot.savenshow("regcoil_vs_lgradb")
        latexplot.figure()
        vs_plot(coil_min_vals, regcoil_vals)
        latexplot.savenshow("regcoil_vs_quasr")



    coil_min_vals = (
        np.array([np.min(coil_surf_dist[f]) for f in bothLgradBandRegcoil]),
        "QUASR coil distance",
    )

    # Do we want to plot the same points as for regcoil?
    # LgradB_vals = (np.array([LgradB_keyed[f] for f in filenames]), "$L^*_{\\nabla B}$") 
    # coil_min_vals = (
    #     np.array([np.min(coil_surf_dist[f]) for f in filenames]),
    #     "QUASR coil distance",
    # )

    if cmap10: 
        plt.rcParams['axes.prop_cycle'] = matplotlib.cycler(color=[plt.get_cmap('tab10')(e) for e in range(len(lgradB_names))])

    #########################
    latexplot.figure()
    vs_plot(coil_min_vals, LgradB_vals, lgradB_names)
    latexplot.savenshow("lgradb_vs_quasr")

    if bdistrib_plot:
        try:
            all_fits = list(map(compare_bdistrib, ids))
            for key in all_fits[0].keys():
                df[key] = list(map(lambda x: x[key], all_fits))
                
            df["QUASR coil distance"] = df["ID"].map({int(k): np.min(v) for k, v in coil_surf_dist.items()})
            df["$L_{REGCOIL}$"] = df["ID"].map({int(k): v for k, v in regcoil_distances.items()}) 
            df["$L^*_{\\nabla B}$"] = df["ID"].map({int(k): (v if len(v) == len(lgradB_names) else np.zeros((len(lgradB_names)))) for k, v in LgradB_keyed.items()}) 
        except Exception as e:
            warnings.warn(e.__str__())
            bdistrib_plot = False

    latexplot.set_cmap(2)
    if bdistrib_plot:
        for reference_qty in ["QUASR coil distance", "$L_{REGCOIL}$"]: #, "$L^*_{\\nabla B}$"
            latexplot.figure()
            subdf = df.dropna(subset=["Bnormal_from_const_v_coils_inductancelog_linear", 
                                      "Bnormal_from_const_v_coils_transferlog_linear",  
                                    #   "Bnormal_from_const_v_coils_transferwindowed_upper_bound",
                                    #   "Bnormal_from_const_v_coils_inductancewindowed_upper_bound",
                                    #   "svd_s_transferMatrixlog_linear",
                                    #   "svd_s_inductanceMatrixlog_linear",
                                      reference_qty])
            subdf = subdf[(subdf[reference_qty] != 0)]
            subdf = subdf[subdf[reference_qty].apply(np.isfinite)]
            #   subdf = subdf[subdf["nc_per_hp"] == 2]
            Bdistrib_vals = (
                list(zip(
                    subdf["Bnormal_from_const_v_coils_transferlog_linear"].tolist(),
                    subdf["Bnormal_from_const_v_coils_inductancelog_linear"].tolist(), 
                    # subdf["svd_s_transferMatrixlog_linear"].tolist(),
                    # subdf["svd_s_inductanceMatrixlog_linear"].tolist(), 
                    )),
                "slope of efficiency sequence $S_e$",
            )
            this_coil_min_vals = (
                subdf[reference_qty].to_numpy(),
                reference_qty,
            )
            vs_plot(
                this_coil_min_vals,
                Bdistrib_vals,
                [
                    "$S_{e, transfer}$ (log-linear fit)",
                    "$S_{e, inductance}$ (log-linear fit)",
                    # "Bnormal_from_1_over_R_field_inductancevalue",
                    # "Bnormal_from_1_over_R_field_inductancewindowed_upper_bound",
                    # "Bnormal_from_1_over_R_field_transfervalue",
                    # "Bnormal_from_1_over_R_field_transferwindowed_upper_bound",
                    # "svd_s_transferMatrixlog_linear",
                    # "svd_s_transferMatrixvalue",
                    # "svd_s_transferMatrixwindowed_upper_bound",
                    # "svd_s_inductance_plasma_middlelog_linear",
                    # "svd_s_inductance_plasma_middlevalue",
                    # "svd_s_inductance_plasma_middlewindowed_upper_bound",
                    # "$S_{e, transfer}$ (windowed max, log-linear fit)",
                    # "$S_{e, inductance}$ (windowed max, log-linear fit)",
                    
                ],
            )
            latexplot.savenshow(reference_qty.replace("$", "").replace("}", "").replace("{", "").replace(" ", "").lower()+"_vs_bdistrib")

In [ ]:
df["$L_{\\nabla |B|}$"] = df["$L^*_{\\nabla B}$"].map(lambda x: 0 if np.any(np.isnan(x)) else x[2])
df[(df["$L_{\\nabla |B|}$"]<5) & (df["QUASR coil distance"]>4) & (df["QUASR coil distance"]<16)]

In [ ]:
# Reproduce LgradB plot from the paper

Regcoil_sep_paper	= (np.array([
	4.1563434578174   ,
4.71329053316564   ,
3.00449439800356   ,
2.18595448826868   ,
1.8644046455127   ,
2.81107988791485   ,
1.36504496725408   ,
1.56411015697888   ,
2.98058875131412   ,
2.87475999236923   ,
2.87502353554767   ,
2.86024762487201   ,
1.3712297354655   ,
0.518894380737076   ,
2.97895380689929   ,
2.60911025351729   ,
1.35447809235562   ,
1.48391717340715   ,
0.662536243901175   ,
1.92476499842764   ,
0.798667400650615   ,
1.79601711501086   ,
1.52055311399254   ,
2.23765551264608   ,
1.65557313612814   ,
0.405954794546203   ,
0.604668693040727   ,
0.618835156186377   ,
2.41021443951025   ,
3.04193891606102   ,
0.877106912488957   ,
0.93736127639918   ,
0.865480331823691   ,
1.41301477568866   ,
1.00149021977749   ,
0.563766071581413   ,
0.409941378143305   ,
1.30094090528035   ,
1.28582877597584   ,
1.381120973084   ,
0.647548252451185   ,
1.59524501762434   ,
0.730208716473146   ,
0.785802668708713   ,
0.991277783910354   ,
]), "$L_{REGCOIL}$")

L_gradB_paper= (np.array([
	6.69926420506388,
	7.53602884697998,
	5.19185883844148,
	3.26102902180622,
	3.42767457700983,
	4.9336093490494,
	2.57224778458006,
	3.0210282808191,
	5.55315224794791,
	5.29773112459343,
	5.30367849202789,
	5.26427362707954,
	2.56325197048168,
	1.25070754843819,
	5.55628137168993,
	3.91183016509675,
	2.73849608882336,
	3.3391608671356,
	1.5811763158286,
	3.05308157060748,
	1.43897341418588,
	3.54180610725063,
	3.18819869694382,
	4.90520097851051,
	1.92586092350114,
	0.969113778064403,
	1.17261904583283,
	1.42140998485691,
	3.66074787826766,
	5.90416097021539,
	1.93433081939344,
	1.6582145953698,
	1.83746396202177,
	3.26340205932931,
	2.32329900159017,
	1.37769555731059,
	1.05376643430081,
	2.61200473419816,
	2.19411538841874,
	3.47153155440648,
	1.33580270544001,
	3.69787656298336,
	1.53543843439843,
	1.72259078097601,
	2.17211739201178,
]), "$L^*_{\\nabla B}$")


latexplot.figure()
vs_plot(
		Regcoil_sep_paper,
		L_gradB_paper
)
latexplot.savenshow("regcoil_vs_lgradb_paper")


# Failiure cases

In [ ]:
# Plot individual VMEC fail
# degenerate = list(filter(lambda x: x[0] == 1301673, results))[0]
latexplot.figure()
import plot_vmec
import importlib
importlib.reload(plot_vmec)
plt.subplot(1, 2, 1)
idx, surfs, coils = get_QUASR_data_sync(1301673)
s = surfs[-1].to_RZFourier()
plot_vmec.plot_vmec(s)
plt.title(f"QUASR 1301673")
plt.subplot(1, 2, 2)
idx, surfs, coils = get_QUASR_data_sync(1101430)
s = surfs[-1].to_RZFourier()

s.scale(0.5)
plot_vmec.plot_vmec(s)
plt.title(f"QUASR 1101430")

latexplot.savenshow("broken_vmec")

# Run failed cases with higher resolution

In [ ]:
vmec = mhd.Vmec(verbose=True)
idx = 923470
# for idx, vals in LgradB_keyed.items():
#   if vals[0] < 0.05:
idx, surfs, coils = get_QUASR_data_sync(int(idx))
vmec.boundary = surfs[-1]
vmec.indata.nfp = vmec.boundary.nfp
vmec_lgradbsc(vmec, idx)

vmec.run()
s = [1]
ntheta = 50
nphi = 50
theta = np.linspace(0, 2 * np.pi, ntheta, endpoint=False)
phi = np.linspace(0, 2 * np.pi / vmec.boundary.nfp, nphi, endpoint=False)
data = vmec_compute_geometry(vmec, s, theta, phi)
B0 = np.mean(data.modB)
print("B0", B0)
vmec.indata.phiedge = 5.865 * vmec.indata.phiedge / B0
vmec.recompute_bell()
vmec.run()
computed = vmec_compute_geometry(vmec, s, theta, phi)
LgradB =  lgradbsc(computed, idx)
print(LgradB )
    

In [ ]:
plt.figure()
plt.subplot(2, 2, 1)
plt.imshow(computed.modB[-1])
plt.colorbar()
plt.subplot(2, 2, 2)


gradB = np.array([[computed.grad_B__XX, computed.grad_B__YX, computed.grad_B__ZX],
                  [computed.grad_B__XY, computed.grad_B__YY, computed.grad_B__ZY],
                  [computed.grad_B__XZ, computed.grad_B__YZ, computed.grad_B__ZZ]])
pltqty = gradB[0,0]
# pltqty = computed.modB / np.linalg.norm(gradB, ord="fro", axis=(0, 1))
plt.imshow(pltqty[0])
plt.colorbar()
plt.savefig("atest.png")

In [ ]:
sys.path.append("/home/missinguser/CSE/single-stage-opt/")
from plots.plot_1soec_kam.plot_boundaries import plot_surf
idx, surfs, coils = get_QUASR_data_sync(2297655)
latexplot.figure()
plot_surf(surfs[-1])
latexplot.savenshow("regcoil_fail_boundaries"+str(idx))